In [1]:
import os
import cv2
from tqdm import tqdm

# ⚙️ Change this to your dataset root
DATASET_ROOT = "."

SPLITS = ["train", "valid", "test"]
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

def is_image_file(filename):
    return os.path.splitext(filename)[1].lower() in IMAGE_EXTS

def check_and_clean_split(split_name):
    images_dir = os.path.join(DATASET_ROOT, split_name, "images")
    labels_dir = os.path.join(DATASET_ROOT, split_name, "labels")

    stats = {
        "processed_images": 0,
        "invalid_images_deleted": 0,
        "orphan_images_deleted": 0,
        "orphan_labels_deleted": 0,
    }

    if not os.path.isdir(images_dir) or not os.path.isdir(labels_dir):
        print(f"[{split_name}] WARNING: 'images' or 'labels' folder missing, skipping.")
        return stats

    image_files = [f for f in os.listdir(images_dir) if is_image_file(f)]
    label_files = [f for f in os.listdir(labels_dir) if f.lower().endswith(".txt")]

    # 1️⃣ Check images with tqdm
    print(f"\n[{split_name}] Checking images...")
    for img_file in tqdm(image_files, desc=f"{split_name} images", ncols=100):
        img_path = os.path.join(images_dir, img_file)
        stem = os.path.splitext(img_file)[0]
        stats["processed_images"] += 1

        img = cv2.imread(img_path)

        # Invalid image → remove image + label
        if img is None:
            try:
                os.remove(img_path)
                stats["invalid_images_deleted"] += 1
            except OSError:
                pass

            label_path = os.path.join(labels_dir, stem + ".txt")
            if os.path.exists(label_path):
                try:
                    os.remove(label_path)
                except OSError:
                    pass
            continue

        # Valid image but missing label → delete image
        label_path = os.path.join(labels_dir, stem + ".txt")
        if not os.path.exists(label_path):
            try:
                os.remove(img_path)
                stats["orphan_images_deleted"] += 1
            except OSError:
                pass

    # 2️⃣ Check labels with tqdm
    print(f"[{split_name}] Checking labels...")
    for lbl_file in tqdm(label_files, desc=f"{split_name} labels", ncols=100):
        stem = os.path.splitext(lbl_file)[0]
        label_path = os.path.join(labels_dir, lbl_file)

        # Check if an image exists with any extension
        img_exists = any(
            os.path.exists(os.path.join(images_dir, stem + ext))
            for ext in IMAGE_EXTS
        )

        if not img_exists:
            try:
                os.remove(label_path)
                stats["orphan_labels_deleted"] += 1
            except OSError:
                pass

    return stats


def main():
    all_stats = {}

    for split in SPLITS:
        print(f"\n=== Processing split: {split} ===")
        stats = check_and_clean_split(split)
        all_stats[split] = stats

    # 🧾 Summary
    print("\n================ SUMMARY ================")
    for split, s in all_stats.items():
        print(f"\n[{split}]")
        print(f"  Images processed           : {s['processed_images']}")
        print(f"  Invalid images deleted     : {s['invalid_images_deleted']}")
        print(f"  Orphan images deleted      : {s['orphan_images_deleted']}")
        print(f"  Orphan labels deleted      : {s['orphan_labels_deleted']}")
    print("\nDone.")

if __name__ == "__main__":
    main()



=== Processing split: train ===

[train] Checking images...


train images: 100%|███████████████████████████████████████████| 21161/21161 [04:05<00:00, 86.04it/s]


[train] Checking labels...


train labels: 100%|█████████████████████████████████████████| 21161/21161 [00:05<00:00, 3575.84it/s]



=== Processing split: valid ===

[valid] Checking images...


...


=== Processing split: test ===

[test] Checking images...


test images: 100%|██████████████████████████████████████████████| 1019/1019 [00:24<00:00, 41.87it/s]


[test] Checking labels...


test labels: 100%|████████████████████████████████████████████| 1019/1019 [00:00<00:00, 3655.62it/s]


================ SUMMARY ================

[train]
  Images processed           : 21161
  Invalid images deleted     : 0
  Orphan images deleted      : 0
  Orphan labels deleted      : 0

[valid]
  Images processed           : 2044
  Invalid images deleted     : 0
  Orphan images deleted      : 0
  Orphan labels deleted      : 0

[test]
  Images processed           : 1019
  Invalid images deleted     : 0
  Orphan images deleted      : 0
  Orphan labels deleted      : 0

Done.
